# Merge for InquiryandPrices Dataset

Initial look at data and data cleaning

## Import necassary libraries

In [ ]:
# Import necassary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from scipy import stats

In [ ]:
#Load datasets
df_prices = pd.read_csv('../data/prices_dataset.csv')

df_inquiries = pd.read_csv('../data/master_inquiries_20210715.csv')

## Correcting date and time formatting.

In [ ]:
df_prices['date_from'] = pd.to_datetime(df_prices['date_from'], errors = 'coerce')
df_prices['date_to'] = pd.to_datetime(df_prices['date_to'], errors = 'coerce')
df_inquiries['date'] = pd.to_datetime(df_inquiries['time'], errors = 'coerce')
df_inquiries['time'] = pd.to_datetime(df_inquiries['date'], errors = 'coerce')
df_inquiries['arrival_date'] = pd.to_datetime(df_inquiries['arrival_date'], errors = 'coerce')
df_inquiries['departure_date'] = pd.to_datetime(df_inquiries['departure_date'], errors = 'coerce')

## Initial look at datasets


In [ ]:
df_inquiries.head()

In [ ]:
df_prices.head() 

## Deleting empty columns

In [ ]:
del df_inquiries['Unnamed: 0']
del df_prices['Unnamed: 0']

## Merging datasets

In [ ]:
df_master = pd.merge(df_inquiries, df_prices, on='listing_id', how='outer', suffixes=('','_key'))
df_master = df_master[(df_master.arrival_date >= df_master.date_from) & (df_master.arrival_date <= df_master.date_to)]
df_master = pd.merge(df_inquiries, df_master, on=[
    'listing_id', 'date'], how='left')
df_master

In [ ]:
df_inquiries.columns

In [ ]:
df_master = df_master.drop(['title_y', 'time_y', 'adult_count_y', 'children_count_y',
       'pets_count_y', 'arrival_date_y', 'departure_date_y', 'inquiry_price_y',
       'length_stay_y'], axis=1)

In [ ]:
df_master.head()

In [ ]:
df_master = df_master.dropna()

In [ ]:
df_master.head()

In [ ]:
df_master.isna().sum()

## Save dataset as CSV


In [ ]:
df_master.to_csv('../data/merged_inquiries_and_prices.csv')